In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

# from textwrap import wrap

%matplotlib inline

# %run cleaning_join_all.ipynb
# df_join_clean = import_df_join_clean() # type: ignore

df_join_clean = pd.read_csv("../data/clean/df_join_clean.csv")

In [ ]:
display(df_join_clean)

In [ ]:
df_join_clean = df_join_clean.sort_values(by=["client_id", "visit_id", "date_time", "process_step" ]).reset_index(drop=True)

df_join_clean

In [ ]:
df_join_clean[df_join_clean["process_step"] == 4].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
df_join_clean[df_join_clean["process_step"] == 0].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate starts for same visit

## Considerations after combining the data:

In [ ]:
# Eveluates if steps are errors

df_join_clean['steps_status'] = None
previous_visit_id = None
previous_value = None

for i in range(len(df_join_clean)):
    current_visit_id = df_join_clean.iloc[i]["visit_id"]
    current_value = df_join_clean.iloc[i]["process_step"]

    if current_value == 0:
        df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "start"
    else:
        if previous_value > current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "error"
        if previous_value == current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "idle"
        if previous_value < current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "ok"
        if current_value == 4:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "confirm"
    
    previous_value = current_value

print(df_join_clean)

### Split the data in the control and test groups

In [ ]:
df_test_group = df_join_clean[df_join_clean["Variation"] == "Test"]
df_control_group = df_join_clean[df_join_clean["Variation"] == "Control"]

df_test_group, df_control_group

### Sort the datasets by: client_id, visit_id (one customer can use the platform several times), process, and datetime


In [ ]:
# # Required:
# # - change process_step to numeral (0 to 4)
# # - evaluate which floats to change to integers
# df_test_group = df_test_group.sort_values(by=["client_id", "visit_id", "process_step", "date_time"]).reset_index(drop=True)
# df_control_group = df_control_group.sort_values(by=["client_id", "visit_id", "process_step", "date_time"]).reset_index(drop=True)

In [ ]:
# Check random client_ids test group

list_of_ids = df_test_group["client_id"].to_list()
df_test_group[df_test_group["client_id"] == random.choice(list_of_ids)][["client_id", "visit_id",  "date_time", "process_step", "steps_status"]]

In [ ]:
# Spot Errors
df_test_group[df_test_group["steps_status"] == "error"]

In [ ]:
# Completion Rate by Group

def get_completion_rate(df):
    num_visits = df["visit_id"].nunique()
    print("Number of Visits >> ", num_visits)

    num_confirm = df[df["process_step"] == 4]
    num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
    num_confirm = num_confirm["process_step"].count()
    print("Number of Confirms >> ", num_confirm)

    completion_rate = (num_confirm / num_visits).round(2)
    print("Completion Rate >> ", completion_rate)

print("Test Group")
get_completion_rate(df_test_group)
print("")

print("Test Group bal > 1000000")
get_completion_rate(df_test_group[df_test_group["bal"] > 1000000])
print("")

print("Test Group bal > 1000000 and Male")
get_completion_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M")])
print("")

print("Test Group bal > 1000000 and Adult Males")
get_completion_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M") & (df_test_group["age_group"] == "Adults") ])
print("")

In [ ]:
print("Control Group")
get_completion_rate(df_control_group)
print("")

print("Control Group bal > 1000000")
get_completion_rate(df_control_group[df_control_group["bal"] > 1000000])
print("")

print("Control Group bal > 1000000 and Male")
get_completion_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M")])
print("")

print("Control Group bal > 1000000 and Adult Males")
get_completion_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M") & (df_control_group["age_group"] == "Adults") ])
print("")

### Determine which percentage of customers in any step of process are hitting errors for control, and test groups.
- Some clients will hit errors in any step of the process and they will go to a previous step. 

In [ ]:
# Error Rate by Group

def get_error_rate(df):
    num_visits = df["visit_id"].nunique()
    print("Number of Visits >> ", num_visits)

    num_steps = df["process_step"].count()
    print("Number of Steps >> ", num_steps)

    num_errors = df[df["steps_status"] == "error"]
    # num_errors.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
    num_errors = num_errors["steps_status"].count()
    print("Number of Errors >> ", num_errors)

    error_rate = (num_errors / num_visits).round(2)
    print("Error Rate per Visit >> ", error_rate)

    error_rate_step = (num_errors / num_steps).round(2)
    print("Error Rate per Steps >> ", error_rate_step)

print("Test Group")
get_error_rate(df_test_group)
print("")

print("Control Group")
get_error_rate(df_control_group)
print("")

print("Test Group bal > 1000000")
get_error_rate(df_test_group[df_test_group["bal"] > 1000000])
print("")

print("Control Group bal > 1000000")
get_error_rate(df_control_group[df_control_group["bal"] > 1000000])
print("")

print("Test Group bal > 1000000 and Male")
get_error_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M")])
print("")

print("Control Group bal > 1000000 and Male")
get_error_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M")])
print("")


- If a customer has several start times, only the last one must be considered.
- If a customer hits the confirmation step several times, consider the last one as the end of the process
- Keep in mind that this is a funnel process. Therefore, not all the customers who start the process will finish it and be able to proceed to the next one.
- There are lots of customers who participated in the experiment over a limited period of time. Some of them will be faster or more successful than others, but all of them count. Therefore, we're looking for "mean" times or rates!!!




## Accordingly:
- Within the same group (control or test), are the completion rates (number of customers who reached the end of a step out of total who started) in each step the same? (... test)
- Between groups (control and test), are the completion rates (number of customers who reached the end of a step out of total who started) in each step the same? (... test)
- Within the same group (control or test), are the completion times of all the steps of the process the same? (... test)
- Between groups (control and test), are the completion times of each step the same? (...step)
- Is the error rate (number of customers who had to go to a previous step out of total) in the test group smaller than the error rate in the control group (whatever error rate is) - 5%?
- Do you find differences by age groups in control and test groups?
- Use Tableau to create graphs to summarize your findings.